In [1]:
import glob
import numpy as np 
import pandas as pd

# 讀入資料

In [2]:
data=pd.read_csv('/Users/chenjessie/Desktop/經濟預測模型/data/全部.csv')
data

,日期,新屋銷售,新屋開工,成屋銷售,CPI,coreCPI,個人支出,信心,失業率,GDP,predict
0,2020-02-01,76.4,156.7,546,0.1,0.2,0.2,101.000000,3.6,2.10000,3
1,2020-01-01,69.4,160.8,554,0.2,0.1,0.3,99.800000,3.5,2.10000,1
2,2019-12-01,71.9,136.5,535,0.3,0.2,0.4,99.300000,3.5,2.10000,1
3,2019-11-01,73.3,131.4,546,0.4,0.2,0.3,96.800000,3.6,2.10000,2
4,2019-10-01,70.1,125.6,538,0.0,0.1,0.2,95.500000,3.5,1.90000,2
...,...,...,...,...,...,...,...,...,...,...,...
597,1970-05-01,44.5,126.0,151,0.5,0.7,0.6,85.917625,4.6,-0.25408,3
598,1970-04-01,38.9,132.0,137,0.5,0.8,-0.3,85.917625,4.4,-0.25408,2
599,1970-03-01,37.3,131.0,146,0.5,0.5,0.8,78.100000,4.2,-0.25408,2
600,1970-02-01,46.1,109.0,153,0.5,0.5,1.0,85.917625,3.9,-0.25408,1


# 切割成train,test

In [3]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(data[['新屋銷售','新屋開工','成屋銷售','CPI','coreCPI','個人支出','信心','失業率','GDP']],data[['predict']],test_size=0.3,random_state=0)

# 標準化x值

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

# SVM
- sklearn.svm.SVC(C=1.0, kernel=’rbf’, degree=3, gamma=0.0, coef0=0.0, shrinking=True, probability=False,tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=None)

## 1. linear

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svm = SVC(kernel='linear', probability = True)  #linear/rbf/poly
svm.fit(x_train_std,y_train['predict'].values)
x_test_result = svm.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test, x_test_result))

Accuracy: 0.861878453038674


## 2. poly


In [6]:
from sklearn.svm import SVC
svm = SVC(kernel='poly', probability = True)  #linear/rbf/poly
svm.fit(x_train_std,y_train['predict'].values)
x_test_result1 = svm.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test, x_test_result1))

Accuracy: 0.856353591160221


## 3. rbf

In [7]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', probability = True)  #linear/rbf/poly
svm.fit(x_train_std,y_train['predict'].values)
x_test_result2 = svm.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test, x_test_result2))

Accuracy: 0.861878453038674


# Decision Tree
- DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,max_features=None, max_leaf_nodes=None,min_impurity_split=1e-07, min_samples_leaf=1,min_samples_split=2, min_weight_fraction_leaf=0.0,presort=False, random_state=None, splitter='best')

In [8]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'gini',max_depth=2) 
tree.fit(x_train_std,y_train)
x_test_result3=tree.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test,x_test_result3))

Accuracy: 0.861878453038674


# Random Forest

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
model = BaggingClassifier(tree, n_estimators=60, max_samples=0.8,)

model.fit(x_train_std,y_train)
x_test_result4=model.predict(x_test_std)
print('Accuracy:',accuracy_score(y_test,x_test_result4))

Accuracy: 0.8453038674033149


/opt/anaconda3/envs/env1/lib/python3.7/site-packages/sklearn/ensemble/_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Conclusion
    SVM 的 kernel＝“linear”,"rbf"
    Decision tree 的max_depth = 2
    的準確度為最高，皆為 86.18%